## Experiments for binary treatment effect estimation comparison

In [1]:
import sys, os

# add the project root to sys.path
root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if root not in sys.path:
    sys.path.insert(0, root)

from data_causl.utils import *
from data_causl.data import *
from frengression import *

device = torch.device('cpu')

import CausalEGM as cegm
# import the module
from models import *

import numpy as np
import pickle
import os
from tqdm import tqdm

from matplotlib import pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)



np.random.seed(42)
n_tr = 1000
n_p = 1000

nI = 2
nX = 2
nO = 2
nS= 2
p = nI+nX+nO+nS
ate = 2
beta_cov = 0
strength_instr = 1
strength_conf = 1
strength_outcome = 1
binary_intervention=True

## Example of hyperparameter tuning

In [10]:
# def make_synthetic_data(N=1000, D=5, seed=0):
#     torch.manual_seed(seed)
#     X = torch.randn(N, D)
#     w_p, b_p = torch.randn(D), 0.1
#     p = torch.sigmoid(X @ w_p + b_p)
#     t = torch.bernoulli(p)
#     beta0 = torch.randn(D)
#     y0 = X @ beta0 + 0.1 * torch.randn(N)
#     tau = (X[:,0] * 2.0).clamp(min=0)
#     y1 = y0 + tau + 0.1 * torch.randn(N)
#     y = y0 * (1 - t) + y1 * t
#     return X.numpy(), t.numpy(), y.numpy(), y0.numpy(), y1.numpy()

# X, t, y, y0, y1 = make_synthetic_data(N=400, D=2)
# X_train, X_tmp, t_train, t_tmp, y_train, y_tmp = train_test_split(X, t, y, test_size=0.4, random_state=42)
# X_val, X_test, t_val, t_test, y_val, y_test = train_test_split(X_tmp, t_tmp, y_tmp, test_size=0.5, random_state=42)

def tune_and_eval(model_name, X_train, t_train, y_train, X_val, t_val, y_val, X_test, t_test, y_test, n_trials=20):
    def objective(trial):
        lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
        wd = trial.suggest_loguniform("wd", 1e-7, 1e-4)
        bs = trial.suggest_categorical("bs", [32, 64, 128])
        epochs = trial.suggest_int("epochs", 50, 200)
        if model_name == "tarnet":
            rep1 = trial.suggest_int("rep1", 50, 200)
            rep2 = trial.suggest_int("rep2", 50, 200)
            head = trial.suggest_int("head", 50, 200)
            drop = trial.suggest_uniform("drop", 0.0, 0.5)
            trainer = TARNetTrainer(X_train.shape[1], [rep1,rep2], [head], drop)
        elif model_name == "cfrnet":
            rep1 = trial.suggest_int("rep1", 50, 200)
            rep2 = trial.suggest_int("rep2", 50, 200)
            head = trial.suggest_int("head", 50, 200)
            drop = trial.suggest_uniform("drop", 0.0, 0.5)
            ipm_w = trial.suggest_loguniform("ipm_weight", 0.01, 10.0)
            trainer = CFRNetTrainer(X_train.shape[1], [rep1,rep2], [head], drop, ipm_w)
        elif model_name == "cevae":
            ld = trial.suggest_int("latent_dim", 10, 200)
            hd = trial.suggest_int("hidden_dim", 20, 400)
            nl = trial.suggest_int("num_layers", 2, 5)
            ns = trial.suggest_categorical("num_samples", [10,50,100,200])
            trainer = CEVAETrainer(X_train.shape[1], ld, hd, nl, ns)
        else:  # dragonnet
            sh = trial.suggest_int("shared_hidden", 50, 200)
            oh = trial.suggest_int("outcome_hidden", 50, 200)
            trainer = DragonNetTrainer(X_train.shape[1], sh, oh)
        return trainer.fit(X_train, t_train, y_train, X_val, t_val, y_val,
                            lr=lr, weight_decay=wd, batch_size=bs, epochs=epochs)
    
    study = optuna.create_study(direction="minimize", study_name=f"{model_name}_tune")
    study.optimize(objective, n_trials=n_trials)
    best = study.best_params
    print(f"Best params for {model_name}: {best}")
    # retrain
    X_trn = np.vstack([X_train, X_val])
    t_trn = np.concatenate([t_train, t_val])
    y_trn = np.concatenate([y_train, y_val])
    if model_name == "tarnet":
        trainer = TARNetTrainer(X_trn.shape[1], [best['rep1'],best['rep2']], [best['head']], best['drop'])
    elif model_name == "cfrnet":
        trainer = CFRNetTrainer(X_trn.shape[1], [best['rep1'],best['rep2']], [best['head']], best['drop'], best['ipm_weight'])
    elif model_name == "cevae":
        trainer = CEVAETrainer(X_trn.shape[1], best['latent_dim'], best['hidden_dim'], best['num_layers'], best['num_samples'])
    else:
        trainer = DragonNetTrainer(X_trn.shape[1], best['shared_hidden'], best['outcome_hidden'])
    trainer.fit(X_trn, t_trn, y_trn, X_test, t_test, y_test,
                lr=best['lr'], weight_decay=best['wd'], batch_size=best['bs'], epochs=best['epochs'])
    if model_name == "cevae":
        return trainer.predict(X_test)
    else:
        y0p, y1p = trainer.predict(X_test)
        return y1p - y0p

# # Tune and train each model separately, storing ITEs
# print("Tuning and training TARNet...")
# ite_tarnet = tune_and_eval("tarnet")
# print("TARNet ITE shape:", ite_tarnet.shape)

# print("Tuning and training CFRNet...")
# ite_cfrnet = tune_and_eval("cfrnet")
# print("CFRNet ITE shape:", ite_cfrnet.shape)

# print("Tuning and training CEVAE...")
# ite_cevae = tune_and_eval("cevae")
# print("CEVAE ITE shape:", ite_cevae.shape)

# print("Tuning and training DragonNet...")
# ite_dragonnet = tune_and_eval("dragonnet")
# print("DragonNet ITE shape:", ite_dragonnet.shape)

# # Now you have separate ITE arrays:
# # ite_tarnet, ite_cfrnet, ite_cevae, ite_dragonnet
# print("All models complete.")


In [15]:
from functools import lru_cache

def tune_and_eval(model_name,
                  X_train, t_train, y_train,
                  X_val,   t_val,   y_val,
                  X_test,  t_test,  y_test,
                  best_params=None,
                  n_trials=20):
    """
    If best_params is None: runs Optuna, returns (ITE_array, best_params).
    If best_params is given: skips Optuna, returns ITE_array only.
    """
    # 1) hyperparam search
    if best_params is None:
        import optuna
        study = optuna.create_study(direction="minimize",
                                    study_name=f"{model_name}_tune")
        def objective(trial):
            # common
            lr     = trial.suggest_loguniform("lr", 1e-5, 1e-2)
            wd     = trial.suggest_loguniform("wd", 1e-7, 1e-4)
            bs     = trial.suggest_categorical("bs", [32, 64, 128])
            epochs = trial.suggest_int("epochs", 50, 200)

            # model‐specific
            if model_name == "tarnet":
                rep1 = trial.suggest_int("rep1", 50, 200)
                rep2 = trial.suggest_int("rep2", 50, 200)
                head = trial.suggest_int("head", 50, 200)
                drop = trial.suggest_uniform("drop", 0.0, 0.5)
                trainer = TARNetTrainer(X_train.shape[1], [rep1,rep2], [head], drop)

            elif model_name == "cfrnet":
                rep1   = trial.suggest_int("rep1", 50, 200)
                rep2   = trial.suggest_int("rep2", 50, 200)
                head   = trial.suggest_int("head", 50, 200)
                drop   = trial.suggest_uniform("drop", 0.0, 0.5)
                ipm_w  = trial.suggest_loguniform("ipm_weight", 0.01, 10.0)
                trainer = CFRNetTrainer(X_train.shape[1], [rep1,rep2], [head], drop, ipm_w)

            elif model_name == "cevae":
                ld = trial.suggest_int("latent_dim", 10, 200)
                hd = trial.suggest_int("hidden_dim", 20, 400)
                nl = trial.suggest_int("num_layers", 2, 5)     # note: 2→5 to avoid pop error
                ns = trial.suggest_categorical("num_samples", [10,50,100,200])
                trainer = CEVAETrainer(X_train.shape[1], ld, hd, nl, ns)

            else:  # dragonnet
                sh = trial.suggest_int("shared_hidden", 50, 200)
                oh = trial.suggest_int("outcome_hidden", 50, 200)
                trainer = DragonNetTrainer(X_train.shape[1], sh, oh)

            return trainer.fit(
                X_train, t_train, y_train,
                X_val,   t_val,   y_val,
                lr=lr, weight_decay=wd,
                batch_size=bs, epochs=epochs
            )

        study.optimize(objective, n_trials=n_trials)
        best_params = study.best_params
        print(f"🔍 Best params for {model_name}: {best_params}")

    # 2) retrain on combined train+val
    X_trn = np.vstack([X_train, X_val])
    t_trn = np.concatenate([t_train, t_val])
    y_trn = np.concatenate([y_train, y_val])

    if model_name == "tarnet":
        trainer = TARNetTrainer(
            X_trn.shape[1],
            [best_params['rep1'], best_params['rep2']],
            [best_params['head']],
            best_params['drop']
        )
    elif model_name == "cfrnet":
        trainer = CFRNetTrainer(
            X_trn.shape[1],
            [best_params['rep1'], best_params['rep2']],
            [best_params['head']],
            best_params['drop'],
            best_params['ipm_weight']
        )
    elif model_name == "cevae":
        trainer = CEVAETrainer(
            X_trn.shape[1],
            best_params['latent_dim'],
            best_params['hidden_dim'],
            best_params['num_layers'],
            best_params['num_samples']
        )
    else:
        trainer = DragonNetTrainer(
            X_trn.shape[1],
            best_params['shared_hidden'],
            best_params['outcome_hidden']
        )

    trainer.fit(
        X_trn, t_trn, y_trn,
        X_test, t_test, y_test,
        lr=best_params['lr'],
        weight_decay=best_params['wd'],
        batch_size=best_params['bs'],
        epochs=best_params['epochs']
    )

    if model_name == "cevae":
        ite = trainer.predict(X_test)
    else:
        y0p, y1p = trainer.predict(X_test)
        ite = y1p - y0p

    return (ite, best_params) if best_params is None else ite


## Fitting synthetic data generated by causl

### Data generation

In [14]:
nrep = 20  # Number of repetitions
n_tr = 1000  # Training sample size
n_val = 400
n_te = 1000  # Testing sample size
strength_instr_values = np.arange(1,4.5,1)  # Varying strength of instrumental variables
nI = 5  # Fixed number of instrumental variables
nX = 5
nO = 0
nS = 0
binary_intervention = True
num_iters = 1000  # Fixed number of training iterations


# Initialize tracker for strength_instr
tracker = {strength_instr: {"fr": [], "dr": [], "causalegm":[], "tarnet":[], "cfrnet":[], "cevae":[], "dragonnet":[]}
           for strength_instr in strength_instr_values}

# Begin loop over strength_instr
for strength_instr in strength_instr_values:
    print(f"Running experiments for strength_instr = {strength_instr}")
    p = nI + nX + nO + nS  # Update the number of covariates
    
    for rep in tqdm(range(nrep)):
        # Generate training and testing data
        df_tr = generate_data_causl(n=n_tr, nI=nI, nX=nX, nO=nO, nS=nS, ate=ate, 
                                    beta_cov=beta_cov, strength_instr=strength_instr, 
                                    strength_conf=strength_conf, 
                                    strength_outcome=strength_outcome, 
                                    binary_intervention=binary_intervention)
        z_tr = torch.tensor(df_tr[[f"X{i}" for i in range(1, p + 1)]].values, dtype=torch.float32)
        x_tr = torch.tensor(df_tr['A'].values, dtype=torch.int32).view(-1, 1) if binary_intervention else \
            torch.tensor(df_tr['A'].values, dtype=torch.float32).view(-1, 1)
        y_tr = torch.tensor(df_tr['y'].values, dtype=torch.float32).view(-1, 1)
        
        z_tr_np = df_tr[[f"X{i}" for i in range(1, p + 1)]].values
        x_tr_np = df_tr['A'].values
        y_tr_np = df_tr['y'].values

        df_val = generate_data_causl(n=n_val, nI=nI, nX=nX, nO=nO, nS=nS, ate=ate, 
                                    beta_cov=beta_cov, strength_instr=strength_instr, 
                                    strength_conf=strength_conf, 
                                    strength_outcome=strength_outcome, 
                                    binary_intervention=binary_intervention)


        z_val_np = df_val[[f"X{i}" for i in range(1, p + 1)]].values
        x_val_np = df_val['A'].values
        y_val_np = df_val['y'].values

        df_te = generate_data_causl(n=n_te, nI=nI, nX=nX, nO=nO, nS=nS, ate=ate, 
                                    beta_cov=beta_cov, strength_instr=strength_instr, 
                                    strength_conf=strength_conf, 
                                    strength_outcome=strength_outcome, 
                                    binary_intervention=binary_intervention)

        z_te_np = df_te[[f"X{i}" for i in range(1, p + 1)]].values
        x_te_np = df_te['A'].values
        y_te_np = df_te['y'].values
        z_te = torch.tensor(z_te_np, dtype=torch.float32)

        model = Frengression(x_dim = x_tr.shape[1], y_dim = 1, z_dim =z_tr.shape[1], 
                             noise_dim=1, num_layer=3, hidden_dim=100, 
                             device=device, x_binary=binary_intervention, z_binary_dims=0)

        # Train Frengression model
        model.train_y(x=x_tr,
                      z=z_tr, 
                      y=y_tr, 
                      num_iters=num_iters, lr=1e-4, print_every_iter=1000)

        # Sample model distributions
        P0 = model.sample_causal_margin(torch.tensor([0], dtype=torch.int32), sample_size=n_te).numpy().reshape(-1, 1)
        P1 = model.sample_causal_margin(torch.tensor([1], dtype=torch.int32), sample_size=n_te).numpy().reshape(-1, 1)
        ate_fr = np.mean(P1) - np.mean(P0)

        # DR Estimation
        ate_dr, _ = dr_ate(x_tr_np, y_tr_np, z_tr_np ,x_te_np, y_te_np, z_te_np)

        ate_tarnet = tune_and_eval("tarnet", z_tr_np, x_tr_np, y_tr_np, z_val_np, x_val_np, y_val_np, z_te_np, x_te_np, y_te_np).mean()
        ate_cfrnet = tune_and_eval("cfrnet", z_tr_np, x_tr_np, y_tr_np, z_val_np, x_val_np, y_val_np, z_te_np, x_te_np, y_te_np).mean()
        ate_cevae = tune_and_eval("cevae", z_tr_np, x_tr_np, y_tr_np, z_val_np, x_val_np, y_val_np, z_te_np, x_te_np, y_te_np).mean()
        ate_dragonnet = tune_and_eval("dragonnet", z_tr_np, x_tr_np, y_tr_np, z_val_np, x_val_np, y_val_np, z_te_np, x_te_np, y_te_np).mean()


        cegm_params = {'dataset': 'Semi_acic', 
                        'output_dir': '.', 
                        'v_dim': z_tr.shape[1], 
                        'z_dims': [1, 1, 1, 1], 
                        'lr': 0.0002, 
                        'alpha': 1, 
                        'beta': 1, 
                        'gamma': 10, 
                        'g_d_freq': 5, 
                        'g_units': [64, 64, 64, 64, 64], 
                        'e_units': [64, 64, 64, 64, 64], 
                        'f_units': [64, 32, 8], 
                        'h_units': [64, 32, 8], 
                        'dz_units': [64, 32, 8], 
                        'dv_units': [64, 32, 8], 'save_res': False, 'save_model': False, 'binary_treatment': True, 'use_z_rec': True, 'use_v_gan': True}
        egm_model = cegm.CausalEGM(params=cegm_params, random_seed=42)
        egm_model.train(data=[x_tr,y_tr,z_tr],n_iter=1000, verbose=False)
        ate_causalegm=egm_model.getCATE(z_te).mean()

        # Log results
        tracker[strength_instr]["fr"].append(ate_fr)
        tracker[strength_instr]["dr"].append(ate_dr)
        tracker[strength_instr]["causalegm"].append(ate_causalegm)
        tracker[strength_instr]['tarnet'].append(ate_tarnet)
        tracker[strength_instr]["cfrnet"].append(ate_cfrnet)
        tracker[strength_instr]["cevae"].append(ate_cevae)
        tracker[strength_instr]["dragonnet"].append(ate_dragonnet)
        


Running experiments for strength_instr = 1.0


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1: loss 2.3320,	loss_y 1.5966, 1.6052, 0.0173,	loss_eta 0.7354, 0.7768, 0.0827
Epoch 1000: loss 0.8561,	loss_y 0.2806, 0.5649, 0.5685,	loss_eta 0.5755, 1.1212, 1.0914


[I 2025-04-25 15:39:29,755] A new study created in memory with name: tarnet_tune
[I 2025-04-25 15:39:31,924] Trial 0 finished with value: 1.5756584405899048 and parameters: {'lr': 1.0973120437415867e-05, 'wd': 9.212781724158886e-06, 'bs': 64, 'epochs': 141, 'rep1': 103, 'rep2': 116, 'head': 174, 'drop': 0.4570287036947749}. Best is trial 0 with value: 1.5756584405899048.
[I 2025-04-25 15:39:34,172] Trial 1 finished with value: 0.7824318408966064 and parameters: {'lr': 0.0008491995663938662, 'wd': 1.3419520420147572e-07, 'bs': 32, 'epochs': 97, 'rep1': 103, 'rep2': 189, 'head': 113, 'drop': 0.037699885483886975}. Best is trial 1 with value: 0.7824318408966064.
[I 2025-04-25 15:39:35,357] Trial 2 finished with value: 0.8074317574501038 and parameters: {'lr': 0.0017772727559814553, 'wd': 7.331689797117618e-05, 'bs': 128, 'epochs': 125, 'rep1': 71, 'rep2': 75, 'head': 121, 'drop': 0.13604197627169107}. Best is trial 1 with value: 0.7824318408966064.
[I 2025-04-25 15:39:37,993] Trial 3 fini

Best params for tarnet: {'lr': 0.0001818238381157221, 'wd': 1.1705514819485741e-07, 'bs': 64, 'epochs': 164, 'rep1': 115, 'rep2': 188, 'head': 145, 'drop': 0.09667685880554794}


...

Best params for cfrnet: {'lr': 0.001536424148255225, 'wd': 3.1116100144714286e-06, 'bs': 64, 'epochs': 84, 'rep1': 200, 'rep2': 105, 'head': 101, 'drop': 0.08654230197953175, 'ipm_weight': 0.24040708090079768}


[I 2025-04-25 16:17:00,107] A new study created in memory with name: cevae_tune
INFO 	 Training with 8 minibatches per epoch
[I 2025-04-25 16:17:08,474] Trial 0 finished with value: 33.61634436035156 and parameters: {'lr': 0.0014609966844419547, 'wd': 1.2021056360766675e-07, 'bs': 128, 'epochs': 127, 'latent_dim': 79, 'hidden_dim': 138, 'num_layers': 4, 'num_samples': 10}. Best is trial 0 with value: 33.61634436035156.
INFO 	 Training with 8 minibatches per epoch
[I 2025-04-25 16:17:15,240] Trial 1 finished with value: 28.19489727783203 and parameters: {'lr': 0.0004884560555155418, 'wd': 6.603465705577309e-06, 'bs': 128, 'epochs': 120, 'latent_dim': 13, 'hidden_dim': 110, 'num_layers': 4, 'num_samples': 200}. Best is trial 1 with value: 28.19489727783203.
INFO 	 Training with 16 minibatches per epoch
[I 2025-04-25 16:17:25,596] Trial 2 finished with value: 35.6038804321289 and parameters: {'lr': 6.646348681165718e-05, 'wd': 5.206488787846745e-05, 'bs': 64, 'epochs': 118, 'latent_dim': 

Best params for cevae: {'lr': 0.0037753217809897537, 'wd': 2.4812280333305376e-05, 'bs': 128, 'epochs': 196, 'latent_dim': 10, 'hidden_dim': 80, 'num_layers': 4, 'num_samples': 50}


INFO 	 Evaluating 1 minibatches
[I 2025-04-25 16:21:26,598] A new study created in memory with name: dragonnet_tune
[I 2025-04-25 16:21:28,128] Trial 0 finished with value: 0.31154248118400574 and parameters: {'lr': 8.898823779210125e-05, 'wd': 1.0630117787061903e-07, 'bs': 32, 'epochs': 56, 'shared_hidden': 113, 'outcome_hidden': 143}. Best is trial 0 with value: 0.31154248118400574.
[I 2025-04-25 16:21:30,037] Trial 1 finished with value: 0.2972985506057739 and parameters: {'lr': 0.007921453322429859, 'wd': 3.072394584879721e-06, 'bs': 32, 'epochs': 72, 'shared_hidden': 101, 'outcome_hidden': 112}. Best is trial 1 with value: 0.2972985506057739.
[I 2025-04-25 16:21:32,037] Trial 2 finished with value: 0.29816800355911255 and parameters: {'lr': 0.00018713007979511337, 'wd': 1.5162708963614789e-07, 'bs': 128, 'epochs': 152, 'shared_hidden': 200, 'outcome_hidden': 185}. Best is trial 1 with value: 0.2972985506057739.
[I 2025-04-25 16:21:34,908] Trial 3 finished with value: 0.30009272694

KeyboardInterrupt: 

In [72]:
egm_model.getCATE(z_tr).mean()

2025-04-25 15:02:38.511931: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}
2025-04-25 15:02:38.512307: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),

1.8659396

In [25]:
np.mean(df_tr['y'][df_tr['A']==1])

2.400059767345967

In [47]:
model = Frengression(x_dim = x_tr.shape[1], y_dim = 1, z_dim =z_tr.shape[1], 
                             noise_dim=1, num_layer=3, hidden_dim=100, 
                             device=device, x_binary=binary_intervention, z_binary_dims=0)

In [51]:
model.train_y(x=x_tr,z=z_tr, 
                      y=y_tr, 
                      num_iters=1000, lr=1e-4, print_every_iter=1000)

Epoch 1: loss 1.8026,	loss_y 1.1675, 1.3866, 0.4383,	loss_eta 0.6351, 0.8774, 0.4847
Epoch 1000: loss 0.8402,	loss_y 0.2793, 0.5600, 0.5615,	loss_eta 0.5610, 1.1058, 1.0896


In [73]:
model.sample_causal_margin(torch.tensor([1], dtype=torch.int32), sample_size=n_p).mean()

AttributeError: module 'CausalEGM.model' has no attribute 'sample_causal_margin'

In [53]:
model.sample_causal_margin(torch.tensor([0], dtype=torch.int32), sample_size=n_p).mean()

tensor(0.0739)

In [38]:
y_tr.shape

torch.Size([1000, 1])